In [2]:
import os
import openai
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load environment variables from the parent directory
load_dotenv(dotenv_path=os.path.join("../.env"))

# Retrieve the API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set. Check your .env file.")

client = openai.OpenAI(api_key=OPENAI_API_KEY) 

In [3]:
conn = psycopg2.connect(
    dbname="postgres", 
    user="root", 
    password="password1", 
    host="localhost", 
    port="5432"
)

# Load data into a DataFrame
query = 'SELECT * FROM "Issue"'
df = pd.read_sql(query, conn)

# Define column mapping
column_mapping = {
    "Summary": "summary",
    "Description": "description",
    "Comments": "comments"
}
df.rename(columns=column_mapping, inplace=True)

/tmp/ipykernel_87866/3541435462.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
prompts = f'''
       You are a business analyst assistant helping to review IT service ticket data for a non-profit food bank. Each ticket includes an initial desciption that starts the interaction and a  "comments" field written by users and IT help.
It has the idescribing their experience, frustration, praise, or suggestions.

Analyze the following comments to extract meaningful business insights. Focus on identifying:

Common themes or recurring problems

User sentiments (positive, negative, neutral)

Suggestions or implicit requests for improvement

Any operational inefficiencies or team feedback that could help improve service

Respond with a concise summary of 20 insights that could inform decision-making, followed by specific examples if applicable.
Context:

{df[['description', 'comments']]}
'''



In [8]:
response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You are an IT support business analyst."},
                      {"role": "user", "content": prompts}],
        )
        
        # Extract category and subcategory for each response
content = response.choices[0].message.content.strip()

In [10]:
for i in content.split('\n\n'):
    print(i)
    print('-'*100)

Based on the provided service ticket data, here are 20 insights that could inform decision-making for the non-profit food bank:
----------------------------------------------------------------------------------------------------
### Common Themes or Recurring Problems
1. **Cancellation Requests**: Several tickets are related to users needing to cancel orders, indicating potential confusion around the order process or system.
   - *Example*: Users frequently mention they have sent emails to cancel, but may not receive timely confirmations.
----------------------------------------------------------------------------------------------------
2. **Request for Additional Supplies**: Users commonly request to add cases of specific supplies to their orders.
   - *Example*: "Can you please add 25 cases of the following to my order?" highlights a need for efficient order adjustment processes.
----------------------------------------------------------------------------------------------------
3. 

In [11]:
prompts = f'''
       You are a data scientist helping to review IT service ticket data for a non-profit food bank. 
       Each ticket includes an initial desciption that starts the interaction and a  "comments" field written by users and IT help.
       Using the columns and data below, come up with a plan to create priority scores for each ticket.
       These will then used to elevate tickets that need immediate attention.
       Create the scoreing, then create a prompt that can be used by a gpt model to score the tickets.

{df.head(100)}
'''



In [12]:
response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You are an IT support business analyst."},
                      {"role": "user", "content": prompts}],
        )
        
        # Extract category and subcategory for each response
content = response.choices[0].message.content.strip()

In [13]:
content = response.choices[0].message.content.strip()

In [15]:
for i in content.split('\n\n'):
    print(i)
    print('-'*100)

To create a priority scoring system for the IT service tickets at a non-profit food bank, we need to outline a method to evaluate each ticket based on several critical factors. The goal is to identify which tickets require immediate attention and those that can be addressed later.
----------------------------------------------------------------------------------------------------
### Step 1: Define the Scoring Criteria
We will develop a scoring system based on various parameters within the ticket data. Below are the potential criteria that can be used to assess the urgency and importance of each ticket:
----------------------------------------------------------------------------------------------------
1. **Summary Keywords**: Identify keywords in the summary that indicate urgency (e.g., "urgent," "important," "tomorrow," "ASAP"). Each keyword can add to the score.
   
2. **Comment Analysis**: Analyze comments for language signaling urgency. Negative sentiment or expressions of frustra

In [23]:
import re
from textblob import TextBlob
from datetime import datetime
def score_ticket(row, assignee_ticket_counts):
    score = 0
    if row['status'].lower() == 'closed':
        return 0
    # 1. Summary keywords
    keywords = [
    "urgent", "immediate", "important", "asap", "critical", "emergency",
    "high priority", "unusable", "broken", "system down", "not working",
    "failure", "crashed", "inaccessible", "deadline", "today", "tomorrow",
    "locked out", "can't login", "issue needs resolution", "need now",
    "priority", "pressing", "as soon as possible", "need access",
    "blocking", "stuck", "outage", "halted", "failure to load", "can't connect",
    "high severity", "server down", "production issue", "needs fixing now", 'now', 
    
]
    score += sum(2 for kw in keywords if kw in row['summary'].lower())
    
    # 2. Comments sentiment
    if pd.notna(row['comments']):
        blob = TextBlob(row['comments'])
        if blob.sentiment.polarity < 0:
            score += 3
    
    # 3. Time to resolution (if still open)
    if row['status'].lower() == 'open':
        days_open = (datetime.now() - pd.to_datetime(row['created'])).days
        if days_open > 7:
            score += days_open - 7

    # 4. Status
    if row['status'].lower() == 'open':
        score += 5

    # 5. Project category
    if str(row.get('priority', '')).lower() == 'critical':
        score += 4

    # 6. Assignee workload
    if assignee_ticket_counts.get(row['assignee'], 0) > 10:
        score += 2

    # 7. Ticket age
    created_date = pd.to_datetime(row['created'])
    weeks_old = (datetime.now() - created_date).days // 7
    score += weeks_old

    return score
open_tickets = df[df['status'].str.lower() == 'open']
assignee_ticket_counts = open_tickets['assignee'].value_counts().to_dict()
df['formula_priority'] = df.apply(lambda row: score_ticket(row, assignee_ticket_counts), axis=1)



In [26]:
df.sort_values(by='formula_priority', ascending=False, inplace=True)
for i in df[['summary', 'description', 'formula_priority']].itertuples():
    print(i.summary)
    print(i.description)
    print(i.formula_priority)
    print('-'*100)

Refund/Return SO156281
Good morning,



This is to let you know that we received the above mentioned last Friday, 4/26/2024, that included chicken drumsticks, item #300435, but the total amount, $1,748, was not covered by the grant, only $1,639.06 was covered by the grant. Please any idea what might have caused this since we had enough grant money to cover the total amount at the time of order?



Thank you,

Kunle
50
----------------------------------------------------------------------------------------------------
Shopping Menu Questions 362014 Eggs, 30 dozen 30/24 oz Carton
Good morning,



Please is this item just liquid egg or regular egg in a shell? I just wanted to be sure.



Thank you.
28
----------------------------------------------------------------------------------------------------
Access to Washington View Account 
Hi,

When I sign into my account, it looks like I only have access to Villages of East River. I need access to Washington View, our newest site. If possible